In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

## Importing the data

In [2]:
raw_data = pd.read_csv('Index2018.csv')
raw_data.head()
# market indexes

,date,spx,dax,ftse,nikkei
0,07/01/1994,469.90,2224.95,3445.98,18124.01
1,10/01/1994,475.27,2225.00,3440.58,18443.44
2,11/01/1994,474.13,2228.10,3413.77,18485.25
3,12/01/1994,474.17,2182.06,3372.02,18793.88
4,13/01/1994,472.47,2142.37,3360.01,18577.26


## Transform raw data into Time-Series Data

In [3]:
df = raw_data.copy()
df.date = pd.to_datetime(df.date, dayfirst = True)
df.set_index('date', inplace = True)
df = df.asfreq('b') # 'b' - business days
df.fillna(method = 'ffill', inplace = True)
df.head()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25
1994-01-12,474.17,2182.06,3372.02,18793.88
1994-01-13,472.47,2142.37,3360.01,18577.26


In [4]:
df.isna().sum()

spx       0
dax       0
ftse      0
nikkei    0
dtype: int64

## Creating Returns

In [5]:
# create returns out of prices
# returns = (p(t) - p(t-1)) / p(t-1)
df['ftse_returns'] = df.ftse.pct_change(1).mul(100)
df.head()

,spx,dax,ftse,nikkei,ftse_returns
date,,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01,NaN
1994-01-10,475.27,2225.00,3440.58,18443.44,-0.156704
1994-01-11,474.13,2228.10,3413.77,18485.25,-0.779229
1994-01-12,474.17,2182.06,3372.02,18793.88,-1.222988
1994-01-13,472.47,2142.37,3360.01,18577.26,-0.356166


## Simple GARCH(1, 1) model

In [6]:
from arch import arch_model

garch_1 = arch_model(df.ftse_returns.iloc[1:], 
                    mean = 'Constant', 
                    vol = 'GARCH', 
                    p = 1,
                    q = 1)

results_garch_1 = garch_1.fit()
results_garch_1.summary()

Iteration:      1,   Func. Count:      6,   Neg. LLF: 13095358228.925074
Iteration:      2,   Func. Count:     15,   Neg. LLF: 3316272663.839606
Iteration:      3,   Func. Count:     23,   Neg. LLF: 8471.10851526324
Iteration:      4,   Func. Count:     29,   Neg. LLF: 8478.252806125416
Iteration:      5,   Func. Count:     35,   Neg. LLF: 8418.145534541141
Iteration:      6,   Func. Count:     41,   Neg. LLF: 8503.158736023322
Iteration:      7,   Func. Count:     47,   Neg. LLF: 8418.153230157168
Iteration:      8,   Func. Count:     53,   Neg. LLF: 8417.890524627057
Iteration:      9,   Func. Count:     59,   Neg. LLF: 8410.647245530905
Iteration:     10,   Func. Count:     64,   Neg. LLF: 8410.646877975418
Iteration:     11,   Func. Count:     69,   Neg. LLF: 8410.646871498702
Iteration:     12,   Func. Count:     73,   Neg. LLF: 8410.646871496969
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8410.646871498702
            Iterations: 12
 

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:           ftse_returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -8410.65
Distribution:                  Normal   AIC:                           16829.3
Method:            Maximum Likelihood   BIC:                           16856.3
                                        No. Observations:                 6276
Date:                Sat, Nov 19 2022   Df Residuals:                     6275
Time:                        03:03:10   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0422  1.030e-02      4.103  4.081e-05 [2.206e-02,6.243e-02]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0124  3.264e-03      3.810  1.391e-04 [6.037e-03,1.883e-02]
alpha[1]       0.0880  1.135e-02      7.749  9.242e-15   [6.571e-02,  0.110]
beta[1]        0.9018  1.279e-02     70.518      0.000     [  0.877,  0.927]
============================================================================

Covariance estimator: robust
"""

## Higher-Lag GARCH Models

It is proven that no higher-order GARCH models outperform the GARCH(1, 1) when it comes to variance of market returns.<br/>
So it is no need to include more than 1 GARCH component.

In [7]:
# but still lets try to fit GARCH(1, 2)
garch_bad = arch_model(df.ftse_returns.iloc[1:], 
                    mean = 'Constant', 
                    vol = 'GARCH', 
                    p = 1,
                    q = 2)

results_garch_bad = garch_bad.fit()
results_garch_bad.summary()

# see p-value of beta[2]!
# it happend because of autocorrelation!

Iteration:      1,   Func. Count:      7,   Neg. LLF: 251759562517807.75
Iteration:      2,   Func. Count:     17,   Neg. LLF: 317978720.5780242
Iteration:      3,   Func. Count:     25,   Neg. LLF: 12243.662539799447
Iteration:      4,   Func. Count:     33,   Neg. LLF: 8472.23578195099
Iteration:      5,   Func. Count:     40,   Neg. LLF: 8420.601530652337
Iteration:      6,   Func. Count:     47,   Neg. LLF: 8411.60100635856
Iteration:      7,   Func. Count:     53,   Neg. LLF: 8479.252123336752
Iteration:      8,   Func. Count:     60,   Neg. LLF: 8518.749729243198
Iteration:      9,   Func. Count:     67,   Neg. LLF: 8410.663120877793
Iteration:     10,   Func. Count:     73,   Neg. LLF: 8410.652520595515
Iteration:     11,   Func. Count:     79,   Neg. LLF: 8410.646894040881
Iteration:     12,   Func. Count:     85,   Neg. LLF: 8410.646872951123
Iteration:     13,   Func. Count:     91,   Neg. LLF: 8410.646871423822
Iteration:     14,   Func. Count:     96,   Neg. LLF: 8410.64687

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:           ftse_returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -8410.65
Distribution:                  Normal   AIC:                           16831.3
Method:            Maximum Likelihood   BIC:                           16865.0
                                        No. Observations:                 6276
Date:                Sat, Nov 19 2022   Df Residuals:                     6275
Time:                        03:03:11   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0422  1.029e-02      4.107  4.012e-05 [2.209e-02,6.241e-02]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0124  3.126e-03      3.977  6.967e-05 [6.307e-03,1.856e-02]
alpha[1]       0.0880  1.354e-02      6.498  8.153e-11   [6.142e-02,  0.114]
beta[1]        0.9018      0.202      4.458  8.256e-06     [  0.505,  1.298]
beta[2]        0.0000      0.193      0.000      1.000     [ -0.378,  0.378]
============================================================================

Covariance estimator: robust
"""

In [8]:
# ok lets try something different

garch_2 = arch_model(df.ftse_returns.iloc[1:], 
                    mean = 'Constant', 
                    vol = 'GARCH', 
                    p = 2,
                    q = 1)

results_garch_2 = garch_2.fit()
results_garch_2.summary()

#alpha[2] coefficient is not significant!

Iteration:      1,   Func. Count:      7,   Neg. LLF: 2180598911.4987574
Iteration:      2,   Func. Count:     17,   Neg. LLF: 601619385.9015592
Iteration:      3,   Func. Count:     25,   Neg. LLF: 12473.673620621077
Iteration:      4,   Func. Count:     33,   Neg. LLF: 8463.73430294025
Iteration:      5,   Func. Count:     40,   Neg. LLF: 10393.303546786376
Iteration:      6,   Func. Count:     47,   Neg. LLF: 8416.60166511925
Iteration:      7,   Func. Count:     54,   Neg. LLF: 8438.445061415607
Iteration:      8,   Func. Count:     62,   Neg. LLF: 8423.018956663976
Iteration:      9,   Func. Count:     69,   Neg. LLF: 8409.731124694044
Iteration:     10,   Func. Count:     75,   Neg. LLF: 8409.73299879124
Iteration:     11,   Func. Count:     82,   Neg. LLF: 8409.7300288558
Iteration:     12,   Func. Count:     89,   Neg. LLF: 8409.729356996137
Iteration:     13,   Func. Count:     95,   Neg. LLF: 8409.729351153039
Iteration:     14,   Func. Count:    100,   Neg. LLF: 8409.7293511

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:           ftse_returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -8409.73
Distribution:                  Normal   AIC:                           16829.5
Method:            Maximum Likelihood   BIC:                           16863.2
                                        No. Observations:                 6276
Date:                Sat, Nov 19 2022   Df Residuals:                     6275
Time:                        03:03:11   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0420  1.033e-02      4.066  4.791e-05 [2.176e-02,6.227e-02]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0140  4.423e-03      3.163  1.559e-03  [5.323e-03,2.266e-02]
alpha[1]       0.0718  1.713e-02      4.193  2.757e-05    [3.825e-02,  0.105]
alpha[2]       0.0231  2.427e-02      0.953      0.341 [-2.445e-02,7.069e-02]
beta[1]        0.8935  1.882e-02     47.470      0.000      [  0.857,  0.930]
=============================================================================

Covariance estimator: robust
"""

In [9]:
garch_3 = arch_model(df.ftse_returns.iloc[1:], 
                    mean = 'Constant', 
                    vol = 'GARCH', 
                    p = 3,
                    q = 1)

results_garch_3 = garch_3.fit()
results_garch_3.summary()

Iteration:      1,   Func. Count:      8,   Neg. LLF: 3512814238.1068425
Iteration:      2,   Func. Count:     19,   Neg. LLF: 3385076830.667379
Iteration:      3,   Func. Count:     28,   Neg. LLF: 12972.93553516673
Iteration:      4,   Func. Count:     37,   Neg. LLF: 8461.105504126728
Iteration:      5,   Func. Count:     45,   Neg. LLF: 15975.825733994367
Iteration:      6,   Func. Count:     53,   Neg. LLF: 8428.769578284342
Iteration:      7,   Func. Count:     61,   Neg. LLF: 8438.88017453031
Iteration:      8,   Func. Count:     69,   Neg. LLF: 8417.43881068167
Iteration:      9,   Func. Count:     77,   Neg. LLF: 8409.835808180063
Iteration:     10,   Func. Count:     84,   Neg. LLF: 8409.785040024888
Iteration:     11,   Func. Count:     91,   Neg. LLF: 8409.730106347826
Iteration:     12,   Func. Count:     98,   Neg. LLF: 8409.729391354598
Iteration:     13,   Func. Count:    105,   Neg. LLF: 8409.729353327373
Iteration:     14,   Func. Count:    112,   Neg. LLF: 8409.72935

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:           ftse_returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -8409.73
Distribution:                  Normal   AIC:                           16831.5
Method:            Maximum Likelihood   BIC:                           16871.9
                                        No. Observations:                 6276
Date:                Sat, Nov 19 2022   Df Residuals:                     6275
Time:                        03:03:12   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0420  1.037e-02      4.051  5.101e-05 [2.169e-02,6.235e-02]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0140  5.541e-03      2.525  1.157e-02  [3.130e-03,2.485e-02]
alpha[1]       0.0718  1.713e-02      4.194  2.745e-05    [3.826e-02,  0.105]
alpha[2]       0.0231  2.448e-02      0.944      0.345 [-2.487e-02,7.110e-02]
alpha[3]   4.4790e-11  2.575e-02  1.739e-09      1.000 [-5.047e-02,5.047e-02]
beta[1]        0.8935  2.601e-02     34.356 1.142e-258      [  0.843,  0.944]
=============================================================================

Covariance estimator: robust
"""

OK, out model is getting worse if we increase any of 'p' or 'q' values.<br/>
The GARCH(1, 1) model is the best model for measuring volatility of returns!